# Time dependent D3D simulation

In [ ]:
using Plots;
gr(format=:svg)
@time using FUSE
using Interact
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
use_local_cache = true

shot = 168830; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache) # default test
#shot = 194306; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache, new_impurity_match_power_rad=:Kr); # negT
#shot = 200000; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache); # ECH",
#shot = 156905; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache); # Weird ZIPFIT holds <-- fast_particles_profiles!
#shot = 163303; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache); # standard OMFIT test case
#shot = 190904; @time ini, act = FUSE.case_parameters(:D3D, shot; fit_profiles=false, use_local_cache, EFIT_tree="EFIT01", ); # high q-min (difficult!)
#shot = 169862; @time ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache);

@checkin :fetch ini act;

In [ ]:
# initialization takes experimental data and populates dd.pulse_schedule in a way that allows us to postdict the experiments
# If we ever wanted to play "what if" simulations in FUSE, we would change the dd.pulse_schedule

@checkout :fetch ini act
dd = IMAS.dd()
@time FUSE.init!(dd, ini, act);
@checkin :init dd act;
dd1 = FUSE.checkpoint[:init].dd;

In [ ]:
@manipulate for time0 in dd.core_profiles.time
    plot(dd1.core_profiles.profiles_1d[Float64(time0)]; charge_exchange=true, thomson_scattering=true)
    #plot(dd.equilibrium.time_slice[time0])
end

In [ ]:
@checkout :init dd act;
experiment_LH = FUSE.LH_analysis(dd; do_plot=true);

In [ ]:
@checkout :init dd act;

act.ActorPedestal.model = :dynamic
act.ActorPedestal.tau_n = experiment_LH.tau_n
act.ActorPedestal.tau_t = experiment_LH.tau_t
act.ActorWPED.ped_to_core_fraction = experiment_LH.W_ped_to_core_fraction
act.ActorEPED.ped_factor = 1.0
act.ActorPedestal.T_ratio_pedestal = 1.0 # Ti/Te in the pedestal

if true
    # density and Zeff from experiment
    act.ActorPedestal.density_ratio_L_over_H = 1.0
    act.ActorPedestal.zeff_ratio_L_over_H = 1.0
else
    # pulse_schedule.density and zeff will contain information about the H-mode time-traces
    # The L-mode density and Zeff are set based on a fixed ratio
    act.ActorPedestal.density_ratio_L_over_H = experiment_LH.ne_L_over_H
    act.ActorPedestal.zeff_ratio_L_over_H = experiment_LH.zeff_L_over_H
    dd.pulse_schedule.density_control.n_e_line.reference = experiment_LH.ne_H
    dd.pulse_schedule.density_control.zeff_pedestal.reference = experiment_LH.zeff_H
end

if false
    # LH-transition from LH scaling law
    act.ActorPedestal.mode_transitions = missing
else
    # LH-transition at user-defined times
    act.ActorPedestal.mode_transitions = experiment_LH.mode_transitions
end

# equilibrium model setting
act.ActorEquilibrium.model = :FRESCO #:EGGO or FRESCO
act.ActorFRESCO.nR = act.ActorFRESCO.nZ = 65
act.ActorEGGO.timeslice_average = 4

act.ActorNeutralFueling.τp_over_τe = 0.25

act.ActorFluxMatcher.evolve_plasma_sources = false
# act.ActorFluxMatcher.rho_transport = 0.3:0.05:0.85
act.ActorFluxMatcher.algorithm = :simple
act.ActorFluxMatcher.max_iterations = -10 # negative to avoid print of warnings
act.ActorFluxMatcher.evolve_pedestal = false
act.ActorFluxMatcher.evolve_Te = :flux_match
act.ActorFluxMatcher.evolve_Ti = :flux_match
act.ActorFluxMatcher.evolve_densities = :flux_match
act.ActorFluxMatcher.evolve_rotation = :replay
act.ActorPedestal.rotation_model = :replay # we don't have a good model for the rotation boundary condition

act.ActorFluxMatcher.relax = 0.5

act.ActorTGLF.tglfnn_model = "sat1_em_d3d"
# act.ActorTGLF.model = :GKNN
# act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"

# setup time
δt = 0.05
dd.global_time = ini.time.simulation_start # start_time should be early in the shot, otherwise ohmic current will be wrong
final_time = ini.general.dd.equilibrium.time[end]
act.ActorDynamicPlasma.Nt = Int(ceil((final_time - dd.global_time) / δt))
act.ActorDynamicPlasma.Δt = final_time - dd.global_time

# choose what to evolve
act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true

# choose what to replay
# "replay" means playback from experiment
# act.ActorCoreTransport.model = :replay
# act.ActorPedestal.model = :replay
# act.ActorCurrent.model = :replay
# act.ActorEquilibrium.model = :replay
# act.ActorHCD.ec_model = :replay
# act.ActorHCD.ic_model = :replay
# act.ActorHCD.lh_model = :replay
# act.ActorHCD.nb_model = :replay
# act.ActorHCD.pellet_model = :replay
# act.ActorHCD.neutral_model = :none

# kinetic equilibrium mode ==> ActorCoreTransport and ActorPedestal model as replay
# ie. use kinetic profiles from experiment
kinetic_equilibrium = false
if kinetic_equilibrium
    act.ActorCoreTransport.model = :replay
    act.ActorPedestal.model = :replay

    act.ActorPFactive.boundary_weight = 1.0
    act.ActorPFactive.magnetic_probe_weight = 1.0
    act.ActorPFactive.flux_loop_weight = 1.0
    act.ActorPFactive.strike_points_weight = 0.0
    act.ActorPFactive.x_points_weight = 1.0
else
    act.ActorPFactive.boundary_weight = 1.0
    act.ActorPFactive.magnetic_probe_weight = 0.0
    act.ActorPFactive.flux_loop_weight = 0.0
    act.ActorPFactive.strike_points_weight = 0.0
    act.ActorPFactive.x_points_weight = 1.0
    
    # it's best to start from a transport simulation that is flux-matched
    FUSE.ActorFluxMatcher(dd, act; verbose=true, evolve_plasma_sources=true, max_iterations=1000, do_plot=false);
end

# # change pulse schedule
# for unit in dd.pulse_schedule.nbi.unit
#     unit.power.reference ./= 5.0
# end
# dd.pulse_schedule.flux_control.i_plasma.reference .*= 0.8

# run time-dependent simulation
@time actor = FUSE.ActorDynamicPlasma(dd, act; verbose=true);

# synthetic diagnostics
FUSE.ActorMagnetics(dd, act)
FUSE.ActorInterferometer(dd, act)

In [ ]:
@manipulate for time0 in slider(dd.core_profiles.time, value=(dd.core_profiles.time[1]+dd.core_profiles.time[end])/2, label="time")
    # plot(dd.core_profiles; time0)
    FUSE.plot_plasma_overview(dd, Float64(time0); dd1, aggregate_hcd=true, rotation_quantity=:sonic)#, min_power=1E4)

#     IMAS.ylim(Dict{Int,Float64}(
#         -2 => 0.0,
#         -3 => 0.0, 3 => 2.0,
#         -4 => 0.0, 4 => 4.0,
#         -5 => 0.0, 5 => 4.0,
#         6 => 1.1E20,
#         7 => 70,

#         -9 => -0.2, 9 => 2.0,
#         -10 => -0.25, 10 => .5,
#         -11 => -0.25, 11 => .5,
#         -12 => -1E20, 12 => 1.0E20,
#         -13 => -0.2, 13 => 0.5,

#         -16 => 0.0, 16 => 0.101,
#         -17 => 0.0, 17 => 0.101,
#         -18 => -2.0E19, 18 => 2.0E19
#         )
#     )
end

In [ ]:
using Printf
times = dd.equilibrium.time[2:2:end]
prog = FUSE.ProgressMeter.Progress(length(times))
a = @animate for (k,time0) in enumerate(times)
    FUSE.ProgressMeter.next!(prog)
    FUSE.plot_plasma_overview(dd, Float64(time0); dd1, aggregate_hcd=true)#, min_power=1E4)

    IMAS.ylim(Dict{Int,Float64}(
        -2 => 0.0,
        -3 => 0.0, 3 => 2.0,
        -4 => 0.0, 4 => 4.0,
        -5 => 0.0, 5 => 4.0,
        6 => 1.1E20,
        7 => 70,

        -9 => -0.2, 9 => 2.0,
        -10 => -0.25, 10 => .5,
        -11 => -0.25, 11 => .5,
        -12 => -1E20, 12 => 1.0E20,
        -13 => -0.2, 13 => 0.5,

        -16 => 0.0, 16 => 0.101,
        -17 => 0.0, 17 => 0.101,
        -18 => -2.0E19, 18 => 2.0E19
        )
    )

    # p=plot(dd1.equilibrium; time0, color=:black, label=" CAKE ")
    # plot!(ddF.equilibrium; time0, label=" FUSE - FRESCO",lw=2)
    # #plot!(ddE.equilibrium; time0, label=" FUSE - EGGO (ML) ", lw=2)
    # plot!(p[1], dd.wall, color=:black, legend_position = :outertop, legendfontsize=11)
    # #plot!(p[1], dd.pulse_schedule.position_control, nothing; time0)
    # plot!(p[1], ylim=(-1.5,1.5))
    # plot!(p[2], ylim=(0.0,0.1))
    # #annotate!(p[2], 0.5, 0.1, text("Pressure [MPa]", 14), title="")
    # #annotate!(p[3], 0.5, 2.2, text("Current [MA/m²]", 14), title="")
    # plot!(p[3], ylim=(0.0,2.3))

    # plot(dd.core_profiles; time0)

#         plot(dd.core_transport; time0)

    # plot(dd.core_sources; time0, aggregate_radiation=true, aggregate_hcd=true, only=5)
end

In [ ]:
# animated gif
if !isdir("D3D_$(shot)")
    mkdir("D3D_$(shot)")
end
gif(a, "D3D_$(shot)/D3D_$(shot).gif", fps=12)

In [ ]:
# running the time dependent simulation as a study (this is how IRI runs FUSE)

using Distributed

sty = FUSE.study_parameters(:Postdictive)
sty.server = "localhost"
sty.n_workers = 0
sty.release_workers_after_run = true
sty.save_folder = mktempdir()
sty.device = :D3D
sty.shots = [168830]
sty.reconstruction = true

sty.kw_case_parameters=Dict{Symbol,Any}(
    :use_local_cache=>true,
    :fit_profiles=>true)

@everywhere import FUSE
@everywhere ProgressMeter = FUSE.ProgressMeter

study = FUSE.StudyPostdictive(sty)
FUSE.run(study)